# Attr-Parameters
This notebook shows how to use `Parameters` which is a dataclass developed using the `attr` package along with all the additional features it provides.

In [12]:
import attr
from pprint import pprint as print
from typing import Union, Optional
from copy import deepcopy

The following need to be imported from `param_impl` module to get full benefit of this framework:

1.  `Parameters`: This (data)class forms the core of the framework. All param classes should subclass this (and additionally add the `@attr.s(auto_atribs=True)` decorator).
2.  `Settings`: A list-type class used to specify multiple values for a parameter for hyper-parameter search. Supports all operations of a regular python `list`.
3.  `default_value`: A function used specify default values when they are mutable eg. list, class objects etc. Refer to [this](https://docs.python-guide.org/writing/gotchas/#mutable-default-arguments) to know why this is important.

In [13]:
from param_impl import Parameters, Settings, default_value

Define the all the parameters as a hierarchy of dataclasses subclassing `Parameters`. Things to note:

1.  Don't forget to add the decorator `@attr.s` and subclass `Parameters`.
2.  For each attribute specify type and default value. If the type is a class, use `default_value()` function to specify it. 
3.  Some attributes may allow multiple types. In this case do the following:
    1.  specify the type as `Union[type1, type2, ..., typek]`
    2.  override the `@classmethod` called `get_disambiguators()` that returns a dictionary with all `Union` types in that dataclass as    
    keys and a "disambiguator" functions as value. Disambiguator functions are functions that takes two inputs, an object and a union type
    corresponding to all possible types that the object can have and returns the actual type of that object. To avoid repetition, specify 
    all disambiguators in the `disambiguate()` function.

The following cell contains examples of the above process.

In [14]:
def disambiguate(o, t): 
    lambdas = {
        Union[AdamOptimizerParams, SGDOptimizerParams]: lambda o, _: SGDOptimizerParams if 'momentum' in o else AdamOptimizerParams,
        Union[int, str]: lambda *_: None
    }
    if t in lambdas:
        return lambdas[t](o, t)
    # elif t == Union[t1, t2, t3]:  # Write disambiguator like this when a simple lambda is not possible
    #     pass
    else:
        raise TypeError("Unknown Type")

@attr.s(auto_attribs=True)
class EncoderParams(Parameters):
    type: str = 'torch.nn.LSTM'
    hidden_size: int = 100
    num_layers: int = 1
    bias: bool = True
    dropout: float = 0
    bidirectional: bool = True


@attr.s(auto_attribs=True)
class ModelParams(Parameters):
    type: str = 'models.simple_tagger.SimpleTagger'
    embedding_param: Union[int, str] = 50
    encoder: Optional[EncoderParams] = None

    @classmethod
    def get_disambiguators(cls):
        return {Union[int, str]: disambiguate}


@attr.s(auto_attribs=True)
class AdamOptimizerParams(Parameters):
    type: str = 'torch.optim.Adam'
    lr: float = 0.001


@attr.s(auto_attribs=True)
class SGDOptimizerParams(Parameters):
    type: str = 'torch.optim.SGD'
    lr: float = 0.001
    momentum: float = 0.1


@attr.s(auto_attribs=True)
class TrainingParams(Parameters):
    num_epochs: int = 20
    optimizer: Union[AdamOptimizerParams,
                     SGDOptimizerParams] = default_value(AdamOptimizerParams())

    @classmethod
    def get_disambiguators(cls):
        return {Union[AdamOptimizerParams, SGDOptimizerParams]: disambiguate}


@attr.s(auto_attribs=True)
class TaggingParams(Parameters):
    random_seed: int = 42
    gpu_idx: int = -1
    model: ModelParams = default_value(ModelParams())
    training: TrainingParams = default_value(TrainingParams())
    
    def __attrs_post_init__(self):
        # this function is called by attr after __init__()
        # useful to modify default values
        pass

In [15]:
params = TaggingParams()
print(params)

TaggingParams(random_seed=42, gpu_idx=-1, model=ModelParams(type='models.simple_tagger.SimpleTagger', embedding_param=50, encoder=None), training=TrainingParams(num_epochs=20, optimizer=AdamOptimizerParams(type='torch.optim.Adam', lr=0.001)))


## Dictionary
`Parameters` can be easily converted to and from dicts as well as flattened dicts. The latter is useful because many packages (eg. comet_ml) do not support nested configurations

In [16]:
# easy conversion to and from dict
print(params.to_dict())
print(TaggingParams.from_dict(params.to_dict()))

{'gpu_idx': -1,
 'model': {'embedding_param': 50,
           'encoder': None,
           'type': 'models.simple_tagger.SimpleTagger'},
 'random_seed': 42,
 'training': {'num_epochs': 20,
              'optimizer': {'lr': 0.001, 'type': 'torch.optim.Adam'}}}
TaggingParams(random_seed=42, gpu_idx=-1, model=ModelParams(type='models.simple_tagger.SimpleTagger', embedding_param=50, encoder=None), training=TrainingParams(num_epochs=20, optimizer=AdamOptimizerParams(type='torch.optim.Adam', lr=0.001)))


In [17]:
# easy conversion to and from flattend dict
print(params.to_flattened_dict())
print(TaggingParams.from_flattened_dict(params.to_flattened_dict()))

{'gpu_idx': -1,
 'model.embedding_param': 50,
 'model.encoder': None,
 'model.type': 'models.simple_tagger.SimpleTagger',
 'random_seed': 42,
 'training.num_epochs': 20,
 'training.optimizer.lr': 0.001,
 'training.optimizer.type': 'torch.optim.Adam'}
TaggingParams(random_seed=42, gpu_idx=-1, model=ModelParams(type='models.simple_tagger.SimpleTagger', embedding_param=50, encoder=None), training=TrainingParams(num_epochs=20, optimizer=AdamOptimizerParams(type='torch.optim.Adam', lr=0.001)))


Equality comparison is supported out of the box (thanks to `attr`). So easy to check desirialising from dictionaries gives the same parameters:

In [18]:
assert TaggingParams.from_dict(params.to_dict()) == params
assert TaggingParams.from_flattened_dict(params.to_flattened_dict()) == params

In [19]:
# Both dict-like and attribute access are supported:
print(params.model.to_dict())
print(params['model'].to_dict())
assert params.model == params['model']

{'embedding_param': 50,
 'encoder': None,
 'type': 'models.simple_tagger.SimpleTagger'}
{'embedding_param': 50,
 'encoder': None,
 'type': 'models.simple_tagger.SimpleTagger'}


In [20]:
# can modify using both dict and attribute access
_params = deepcopy(params)
_params.model.encoder = EncoderParams()
_params['model']['embedding_param'] = 100
print(_params.to_dict())
print(params.to_dict())

{'gpu_idx': -1,
 'model': {'embedding_param': 100,
           'encoder': {'bias': True,
                       'bidirectional': True,
                       'dropout': 0,
                       'hidden_size': 100,
                       'num_layers': 1,
                       'type': 'torch.nn.LSTM'},
           'type': 'models.simple_tagger.SimpleTagger'},
 'random_seed': 42,
 'training': {'num_epochs': 20,
              'optimizer': {'lr': 0.001, 'type': 'torch.optim.Adam'}}}
{'gpu_idx': -1,
 'model': {'embedding_param': 50,
           'encoder': None,
           'type': 'models.simple_tagger.SimpleTagger'},
 'random_seed': 42,
 'training': {'num_epochs': 20,
              'optimizer': {'lr': 0.001, 'type': 'torch.optim.Adam'}}}


## Hyper-parameter Search

### Directly using `Parameters`
`Parameters` can be directly used to specify the values to try out for each parameter and then to get all settings in the grid formed by product of values for each parameter.

In [21]:
params = TaggingParams(model=ModelParams(encoder=EncoderParams()))
print(params.to_dict())

{'gpu_idx': -1,
 'model': {'embedding_param': 50,
           'encoder': {'bias': True,
                       'bidirectional': True,
                       'dropout': 0,
                       'hidden_size': 100,
                       'num_layers': 1,
                       'type': 'torch.nn.LSTM'},
           'type': 'models.simple_tagger.SimpleTagger'},
 'random_seed': 42,
 'training': {'num_epochs': 20,
              'optimizer': {'lr': 0.001, 'type': 'torch.optim.Adam'}}}


Use `Settings` to specify different values for each parameter:


In [27]:
params.model.encoder.hidden_size = Settings([50, 100])
params.training.optimizer.lr = Settings([1e-2, 1e-1])

Now just use the `get_settings()` function to get all the different possible settings:

In [29]:
settings = params.get_settings()
print(len(settings))        # will be equal to the product of the number of values for each parameter
for setting in settings:
    print(setting.to_flattened_dict())

4
{'gpu_idx': -1,
 'model.embedding_param': 50,
 'model.encoder.bias': True,
 'model.encoder.bidirectional': True,
 'model.encoder.dropout': 0,
 'model.encoder.hidden_size': 50,
 'model.encoder.num_layers': 1,
 'model.encoder.type': 'torch.nn.LSTM',
 'model.type': 'models.simple_tagger.SimpleTagger',
 'random_seed': 42,
 'training.num_epochs': 20,
 'training.optimizer.lr': 0.01,
 'training.optimizer.type': 'torch.optim.Adam'}
{'gpu_idx': -1,
 'model.embedding_param': 50,
 'model.encoder.bias': True,
 'model.encoder.bidirectional': True,
 'model.encoder.dropout': 0,
 'model.encoder.hidden_size': 50,
 'model.encoder.num_layers': 1,
 'model.encoder.type': 'torch.nn.LSTM',
 'model.type': 'models.simple_tagger.SimpleTagger',
 'random_seed': 42,
 'training.num_epochs': 20,
 'training.optimizer.lr': 0.1,
 'training.optimizer.type': 'torch.optim.Adam'}
{'gpu_idx': -1,
 'model.embedding_param': 50,
 'model.encoder.bias': True,
 'model.encoder.bidirectional': True,
 'model.encoder.dropout': 0,
 

It is also possible to do the above for attributes of list or any other more complex type.

In [36]:
from typing import List
@attr.s(auto_attribs=True)
class TempParams(Parameters):
    list_param: Optional[List[int]] = None
p = TempParams()
print(p)
p.list_param = Settings([[1], [1,2]])
s = p.get_settings()
print(len(s))        # will be equal to the product of the number of values for each parameter
for _s in s:
    print(_s)

TempParams(list_param=None)
2
TempParams(list_param=[1])
TempParams(list_param=[1, 2])


### Using Raytune without Search Algorithm

In [30]:
from ray import tune 

In [31]:
params = TaggingParams(model=ModelParams(encoder=EncoderParams()))
print(params.to_dict())
params.model.encoder.hidden_size = tune.grid_search([50, 100])
params.training.optimizer.lr = tune.loguniform(1e-3, 1e-1)
print(params.to_flattened_dict())
# Now just pass `params.to_flattened_dict()` as `config` parameter to `tune.run()`.

{'gpu_idx': -1,
 'model': {'embedding_param': 50,
           'encoder': {'bias': True,
                       'bidirectional': True,
                       'dropout': 0,
                       'hidden_size': 100,
                       'num_layers': 1,
                       'type': 'torch.nn.LSTM'},
           'type': 'models.simple_tagger.SimpleTagger'},
 'random_seed': 42,
 'training': {'num_epochs': 20,
              'optimizer': {'lr': [0.01, 0.1], 'type': 'torch.optim.Adam'}}}
{'gpu_idx': -1,
 'model.embedding_param': 50,
 'model.encoder.bias': True,
 'model.encoder.bidirectional': True,
 'model.encoder.dropout': 0,
 'model.encoder.hidden_size': {'grid_search': [50, 100]},
 'model.encoder.num_layers': 1,
 'model.encoder.type': 'torch.nn.LSTM',
 'model.type': 'models.simple_tagger.SimpleTagger',
 'random_seed': 42,
 'training.num_epochs': 20,
 'training.optimizer.lr': <ray.tune.sample.Float object at 0x7f9d9116c370>,
 'training.optimizer.type': 'torch.optim.Adam'}
